In [1]:
# IMPORTS
import pandas as pd
import nltk
import json
import requests 
from korp.korp import Korp

In [3]:
# Give substring to search for. For example "talou." which gets words text involving words like taloudellinen, talous
word_or_substring = 'omena.'

# Give the amount of answers to get
amount_of_texts_to_get = 100

In [4]:
#Define url parameters
definition = {
'defaultcontext'	:"=1+paragraph",
'defaultwithin'		:"=sentence",
'show'				:"=sentence,paragraph,lemma,pos,msd,dephead,deprel,ref,lex,nertag",
'show_struct'		:"=text_title,text_date,text_time,text_sect,text_sub,text_user,sentence_id,text_urlmsg,text_urlboard",
'start'				:"=0",
'end'				:"={}".format(amount_of_texts_to_get), 
'corpus'			:"=S24",
'cqp'				:"=%5Blemma+%3D+%22{}*%22%5D".format(word_or_substring),
'indent'			:"=2"
    }

In [5]:
#Lets construct url
URL= 'https://korp.csc.fi/cgi-bin/korp.cgi?command=query&'
for k,v in definition.items():
    if k == str(list(definition.keys())[-1]):
        URL = URL + str(k) + str(v)
    else:URL = URL + str(k) + str(v) +'&'
    

In [7]:
# sending get request and saving the response as response object 
r = requests.get(url = URL)
  
# extracting data in json format 
data = r.json() 

In [9]:
#Lets get rid of some unneccessary part of the data
data= data['kwic']
# Lets now normalize the data to right format
# It now gets one word per row so not yeat perfect but we will transform data more later
data = pd.io.json.json_normalize(data,
                                  record_path = 'tokens',
                                  meta=[
                                      ['structs', 'sentence_id'],
                                      ['structs', 'text_sect'],
                                      ['structs', 'text_sub'],
                                      ['structs', 'text_time'],
                                      ['structs', 'text_title']
                                       ])#,errors='ignore')

In [11]:
# lets take out unneccessary columns
dataframe_clean = data[[
'structs.sentence_id',
'structs.text_sect',
'structs.text_sub',
'structs.text_time',
'structs.text_title',
'word']]

In [25]:
# Lets check how data looks now
dataframe_clean.head()

,structs.sentence_id,structs.text_sect,structs.text_sub,structs.text_time,structs.text_title,word
0,15277,Ruoka ja juoma,Vegaaniruoka,20:52,"Jouluruoat, juomat ja herkut",Sinun
1,15277,Ruoka ja juoma,Vegaaniruoka,20:52,"Jouluruoat, juomat ja herkut",kannattaisi
2,15277,Ruoka ja juoma,Vegaaniruoka,20:52,"Jouluruoat, juomat ja herkut",ostaa
3,15277,Ruoka ja juoma,Vegaaniruoka,20:52,"Jouluruoat, juomat ja herkut",kunnon
4,15277,Ruoka ja juoma,Vegaaniruoka,20:52,"Jouluruoat, juomat ja herkut",kasvisruokakirja


In [13]:
# Lets fill empty values with __empty__
dataframe_clean = dataframe_clean.fillna(value= '__empty__')

In [14]:
# Combine data so we get one message per row
data_test = pd.DataFrame(dataframe_clean.groupby(['structs.sentence_id',
                                      'structs.text_sect',
                                      'structs.text_sub',
                                      'structs.text_time',
                                      'structs.text_title'])['word'].apply(','.join))

In [17]:
# Lets tokenize data, if you dont have neeeded packages, do the following:
# install nltk with pip
# got to command line in this env, then run python
# Then in python run the following:
# import nltk
# nltk.download() --> This should pop up download window from which you can download needed packages
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
data_test['tokenized'] = 'sanoja'
for row_num in range(len(data_test)):
    tokenized = tokenizer.tokenize(data_test['word'].iloc[row_num])
    data_test['tokenized'].iloc[row_num] = tokenized

In [31]:
data_test = data_test.reset_index()

In [56]:
#print("Amount of unique messages in data {} kpl ".format(data_test['structs.sentence_id'].nunique()))
#print(data_test.groupby('structs.text_sub').size())
#print(data_test.groupby('structs.text_sect').size())

In [57]:
# Save data as csv
data_test.to_csv("data/{}.csv".format(word_or_substring))